In [ ]:
import pandas as pd
from datetime import datetime
import os

In [ ]:
# Paths to your snapshots
snapshots = ["/content/MCA-Insight-Engine/Data/Simulated Snapshots/day1_master.csv",
             "/content/MCA-Insight-Engine/Data/Simulated Snapshots/day2_master.csv",
             "/content/MCA-Insight-Engine/Data/Simulated Snapshots/day3_master.csv"]
# Columns to track for changes
tracked_fields = ["COMPANY_STATUS", "AUTHORIZED_CAP", "PAIDUP_CAPITAL", "COMPANY_NAME"]
output_folder = '/content/MCA-Insight-Engine/Data/Change logs'

In [ ]:
def detect_changes(prev_df, curr_df, date_str):
    changes = []

    prev_cin_set = set(prev_df["CORPORATE_IDENTIFICATION_NUMBER"])
    curr_cin_set = set(curr_df["CORPORATE_IDENTIFICATION_NUMBER"])

    # 1️⃣ New incorporations
    new_cins = curr_cin_set - prev_cin_set
    for cin in new_cins:
        row = curr_df[curr_df["CORPORATE_IDENTIFICATION_NUMBER"] == cin].iloc[0]
        changes.append({
            "CIN": cin,
            "Change_Type": "New Incorporation",
            "Field_Changed": "ALL",
            "Old_Value": None,
            "New_Value": row.to_dict(),
            "Date": date_str
        })

    # 2️⃣ Deregistered companies
    removed_cins = prev_cin_set - curr_cin_set
    for cin in removed_cins:
        row = prev_df[prev_df["CORPORATE_IDENTIFICATION_NUMBER"] == cin].iloc[0]
        changes.append({
            "CIN": cin,
            "Change_Type": "Deregistered",
            "Field_Changed": "ALL",
            "Old_Value": row.to_dict(),
            "New_Value": None,
            "Date": date_str
        })

    # 3️⃣ Field updates
    common_cins = prev_cin_set & curr_cin_set
    for cin in common_cins:
        prev_row = prev_df[prev_df["CORPORATE_IDENTIFICATION_NUMBER"] == cin].iloc[0]
        curr_row = curr_df[curr_df["CORPORATE_IDENTIFICATION_NUMBER"] == cin].iloc[0]

        for field in tracked_fields:
            if prev_row[field] != curr_row[field]:
                changes.append({
                    "CIN": cin,
                    "Change_Type": "Field Update",
                    "Field_Changed": field,
                    "Old_Value": prev_row[field],
                    "New_Value": curr_row[field],
                    "Date": date_str
                })

    return changes

# ----------------------------
# Loop through snapshots to generate change logs
# ----------------------------
for i in range(1, len(snapshots)):
    prev_df = pd.read_csv(snapshots[i-1])
    curr_df = pd.read_csv(snapshots[i])
    date_str = f"Day{i+1}"  # Day2, Day3, etc.

    changes = detect_changes(prev_df, curr_df, date_str)

    # Save changes even if empty
    changes_df = pd.DataFrame(changes)
    output_file = os.path.join(output_folder, f"{date_str}_changes.csv")
    changes_df.to_csv(output_file, index=False)

    print(f"✅ Change log created: {output_file} ({len(changes)} changes detected)")

✅ Change log created: /content/MCA-Insight-Engine/Data/Change logs/Day2_changes.csv (4 changes detected)
✅ Change log created: /content/MCA-Insight-Engine/Data/Change logs/Day3_changes.csv (3 changes detected)
